## Dependencies

In [ ]:
!pip install langchain_core==0.3.24
!pip install langchain_openai==0.2.12

## Clone the Old Definitions

In [3]:
!wget https://raw.githubusercontent.com/MasrourTawfik/Textra_Insights/main/Files/PCG.csv

--2024-12-15 15:59:35--  https://raw.githubusercontent.com/MasrourTawfik/Textra_Insights/main/Files/PCG.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54122 (53K) [text/plain]
Saving to: ‘PCG.csv’

PCG.csv             100%[===================>]  52.85K  --.-KB/s    in 0.001s  

2024-12-15 15:59:35 (39.6 MB/s) - ‘PCG.csv’ saved [54122/54122]



## Implimentation

**token** here is you your Github token, you can get it for free :

- Go to your Github profile.
- Click on **Settings**.
- Click on **Developer settings**.
- Click on **Personal access tokens**.
- Click on **Fine-grained token**.
- Click on **Generate new token**.
- Copy the token.

In [10]:
token = "xxxxxxxxxxxxxxx-xxxxxxxxxxxxxxxxxxx"

In [11]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
import time
from tqdm import tqdm
import pandas as pd

In [12]:
class Refining(BaseModel):
    definition: str = Field("", description="La nouvelle définition clarifiée et concise.")
    examples: str = Field("", description="Une liste de 20 exemples concrets illustrant la définition.")


System_Prompt = """

# Votre objectif

Vous êtes un modèle GPT-4, spécialisé dans l'amélioration et la clarification des définitions.
Votre objectif est de reformuler une définition donnée de manière plus concise, claire et précise, tout en conservant l'intégralité du sens.
Vous devez inclure des exemples spécifiques et variés pour illustrer la définition de manière simple et compréhensible. Chaque exemple doit être pertinent et représentatif des situations courantes liées au terme ou au concept défini.

# Instructions

1. Reformulez la définition de manière plus concise et claire, en utilisant un langage simple et accessible.

2. Ajoutez des exemples pratiques et variés pour illustrer la définition.

3. Ne vous concentrez pas sur des détails techniques ou des termes comptables complexes. L'objectif est d'expliquer les concepts de manière facile à comprendre pour un large public.

4. Incluez au moins 15 exemples courtes concrets pour chaque définition afin de donner une idée précise du concept.

5. Ne pas parler de compte crédité ou de compte débité.
"""

In [13]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", System_Prompt),
        ("human", """
        L'anciene définition est la suivante :
        {Old_Definition}
        """),
    ]
)

LLM = ChatOpenAI(model="gpt-4o-mini",
    #temperature=0,
    api_key=token,
    base_url="https://models.inference.ai.azure.com",
)

LLM_Structered = LLM.with_structured_output(Refining)
LLM_Refiner = prompt | LLM_Structered

def Run_Refiner(definition):
    Input = {"Old_Definition": definition}
    response = LLM_Refiner.invoke(Input)
    return response

def Run_Refiner_All(PCG_dataframe):
    counter = 0
    for index, row in tqdm(PCG_dataframe.iterrows(), total=PCG_dataframe.shape[0], desc="Refining Definitions"):
        old_definition = row["Title"] + "\n\n" + row["Definition"]
        response = Run_Refiner(old_definition)
        new_definition = response.definition
        new_examples = response.examples
        PCG_dataframe.at[index,"Definition"] = new_definition + "\n\n" + new_examples
        # Stop to 1 min after 10 requests
        if counter == 10 :
            print("Pausing for 1 minute...")
            time.sleep(60)
            counter = 0
        # Stop to 2 seconds between each request
        else :
            print("Pausing for 2 seconds...")
            time.sleep(2)
            counter += 1
    PCG_dataframe.to_csv("PCG_Refined.csv", index=False)


## Test

In [14]:
Old_PCG = pd.read_csv("PCG.csv")
Old_PCG.head()

,Id,Title,Definition
0,2111,Frais de constitution,Les frais de constitution correspondent aux fr...
1,2112,Frais préalables au démarrage,Les frais préalables au démarrage corresponden...
2,2113,Frais d'augmentation du capital,Il s'agit des frais engagés lors des opération...
3,2114,"Frais sur opérations de fusions, scissions et ...",Il s'agit des frais engagés à l'occasion des o...
4,2116,Frais de prospection,ll s'agit des frais de recherche de nouveaux c...


In [15]:
Old_PCG = pd.read_csv("PCG.csv")
Run_Refiner_All(Old_PCG)

Refining Definitions:   0%|          | 0/84 [00:00<?, ?it/s]

Pausing for 2 seconds...


Refining Definitions:   1%|          | 1/84 [00:04<05:55,  4.28s/it]

Pausing for 2 seconds...


Refining Definitions:   2%|▏         | 2/84 [00:10<07:14,  5.30s/it]

...

Refining Definitions:  98%|█████████▊| 82/84 [13:50<00:16,  8.20s/it]

Pausing for 2 seconds...


Refining Definitions:  99%|█████████▉| 83/84 [13:57<00:07,  7.72s/it]

Pausing for 2 seconds...


Refining Definitions: 100%|██████████| 84/84 [14:04<00:00, 10.06s/it]
